Vytýkám před závorku: streamovací platformy používají typicky mladší lidi, rádio BY MĚLO BÝT PRO VŠECHNY, měli bychom na to při psaní pamatovat.

In [6]:
import os
import pandas as pd

In [7]:
radio = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))

In [8]:
stream = pd.read_csv(os.path.join("data","CZ_SINGLES_DIGITAL_TOP_100_tydenni.csv"))

In [9]:
stream['rok'].min()

2014

In [10]:
radio = radio[radio['rok'] >= stream['rok'].min()]

## Porovnání výskytů interpretů v top 100 rádio vs. stream od roku 2014

In [11]:
radio.groupby('interpret_hlavni').size().nlargest(10)

interpret_hlavni
Mirai              1171
Kryštof             921
Chinaski            872
Marek Ztracený      820
Slza                748
Ed Sheeran          709
Xindl X             702
Imagine Dragons     657
Robin Schulz        499
Mandrage            486
dtype: int64

In [12]:
radio[radio['interpret_hlavni'] = 'Mirai'].groupby('week')

SyntaxError: cannot assign to subscript here. Maybe you meant '==' instead of '='? (3037739192.py, line 1)

In [ ]:
stream.groupby('interpret_hlavni').size().nlargest(10)

In [ ]:
pomer = (radio.groupby('interpret_hlavni').size() / stream.groupby('interpret_hlavni').size()).sort_values()

### Co lidi streamují víc, než dramaturgové nasazují:

In [ ]:
pomer.head(25)

### Co dramaturgové nasazují víc, než lidi poslouchají:

In [ ]:
pomer.dropna().nlargest(25)

ZDŮRAZŇUJI: jde o prezenci v top 100, ne poměr real vysílání / streamů. Může být zkreslené, nejde o objektivní pořadí, ale dost to napovídá.

### Rodíl nasazovanosti x streamovanosti žánrů

In [ ]:
wikid = pd.read_json(os.path.join("wikidata","wikidata.json"))

In [ ]:
zanry = pd.DataFrame(pomer).rename(columns={0:'nasazovanost'}).merge(wikid[['interpret_hlavni','zanry']], on='interpret_hlavni')

In [ ]:
zanry.explode('zanry').groupby('zanry')['nasazovanost'].mean().dropna().nsmallest(30)

In [ ]:
zanry.explode('zanry').groupby('zanry')['nasazovanost'].mean().dropna().nlargest(30)

Zase: metodologicky je to ošementý, ale obrázek to dělá moc hezkej.

### Co lidi streamují, ale od roku 2014 nebylo v rádových TOP 100:

In [ ]:
streamovani = stream['interpret_hlavni'].drop_duplicates().to_list()
nevysilani = [s for s in streamovani if len(radio[radio['interpret_hlavni'] == s]) == 0]
stream[stream['interpret_hlavni'].isin(nevysilani)].groupby('interpret_hlavni').size().nlargest(60)

DOPÍČI NEMŮŽU UVĚŘIT ŽE TOTO NENÍ CHYBA

## Potenciální námět na domácí úkol: vydestilovat ze všech žánrů pár základních

In [ ]:
len(wikid.explode('zanry')['zanry'].drop_duplicates())

In [ ]:
wikid.explode('zanry')['zanry'].drop_duplicates().to_list()